In [24]:
%matplotlib inline
import matplotlib as plt
import numpy as np
import os
from pandas import DataFrame
import scipy.stats as ss
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

We'd like to build a spam filter using scikit-learn.  This won't
be as sophisticated as modern production filters, but it will be
roughly at the level of Paul Graham's 2002 proposal [A Plan for Spam](http://www.paulgraham.com/spam.html), which gave rise to the
still-useful bogofilter.

One of our goals will be to show off the nice uniform interface
that scikit-learn exports.  At the end we'll see that we can easily
experiment with different classifiers with a minimum of bother.

The data set we used in class, the Enron-Spam corpus, is publicly accessible.  SpamAssassin also offers a public corpus if you want
to explore with even more data.

#Read all the files

Our first step is to read the data into python.  Here we write a
generator function to recursively read each file in a directory,
yielding after each.  This is a common python idiom, since it lets
us use the read_files function easily.  Note that we yield a pair
of items: the file path and the contents of the file.

In [8]:
NEWLINE = b'\n'
SKIP_FILES = set(['cmds'])

def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
        for file_name in file_names:
            if file_name not in SKIP_FILES:
                file_path = os.path.join(root, file_name)
                if os.path.isfile(file_path):
                    past_header, lines = False, []
                    f = open(file_path, 'rb')
                    for line in f:
                        if past_header:
                            lines.append(line)
                        elif line == NEWLINE:
                            past_header = True
                    f.close()
                    yield file_path, NEWLINE.join(lines).decode('cp1252', 'ignore')


#Make a dataframe of the spam.

It is often convenient to manipulate data using pandas DataFrames.  A
DataFrame is very much like a SQL table, and it permits most of the same
sorts of operations, including projection, selection, and joins.

Here we'll iterate over the results of `read_files()` (note how
convenient it is to use by being a generator) and make a DataFrame
with columns `text` and `class`.  Index columns in pandas are special
(being indexes).  Here we index on the filename.

In [6]:
def build_data_frame(path, classification):
    data_frame = DataFrame({'text': [], 'class': []})
    for file_name, text in read_files(path):
        data_frame = data_frame.append(
                DataFrame({'text': [text], 'class': [classification]}, index=[file_name]))
    return data_frame


Now we just need to call our `build_data_frame()` function.  Notice
that we've conveniently organized our spam and ham emails by the paths
where they're stored.  This lets us make a simple table so that we know
the class by the beginning of the filename.

Note that we're assuming you've put the spam directories in a directory
called "data" in the same directory as this notebook.

In [9]:
HAM = 0
SPAM = 1

SOURCES = [
    ('data/spam',         SPAM),
    ('data/easy_ham',     HAM),
    ('data/hard_ham',     HAM),
    ('data/beck-s',       HAM),
    ('data/farmer-d',     HAM),
    ('data/kaminski-v',   HAM),
    ('data/kitchen-l',    HAM),
    ('data/lokay-m',      HAM),
    ('data/williams-w3',  HAM),
    ('data/BG',           SPAM),
    ('data/GP',           SPAM),
    ('data/SH',           SPAM)
    ]

data = DataFrame({'text': [], 'class': []})
for path, classification in SOURCES:
    data = data.append(build_data_frame(path, classification))
  
data = data.reindex(np.random.permutation(data.index))


#The Matrix

Scikit-learn isn't happy unless it is operating on matrices.  So our
first job whenever we use scikit-learn is to convert our data to matrix
form.  In this case, we want to count words in documents, since we are
going to use Bayes Theorem to ask "What is the probability that this new
document is spam given these words in the document?"  Our prior will be the words in the spam corpus we read with `build_data_frame()`.

A `CountVectorizer` object builds a _sparse_ count of the words in the
documents we feed it.  The `fit_and_transform()` function is slightly
more efficient than calling `fit()` and `transform()` separately.  The `fit()` function learns the vocabulary from the raw documents.  If we had
a prior vocabulary, we could potentially use that instead and so not call
`fit()`.  `Transform()` extracts token counts using the vocabulary we
learned from `fit()`.  What gets returned is a numpy sparse matrix.
This is critical, because we probably don't have enough RAM to hold
the dense matrix (that is, with all the zeros represented).

In [11]:
count_vectorizer = CountVectorizer()
# Fit learns the vocabulary.
# Transform does the count.
counts = count_vectorizer.fit_transform(np.asarray(data['text']))


# Naive Bayes Classifier

In machine learning, a classifier is an algorithm that takes data
(representations of objects in the real world) and separates them
into _classes_.  A class is anything, in our case spam or ham.
It is a _supervised_ learning algorithm, which means that we start
by _training_ it on data where we know the classes, and then we
ask it to classify data where we don't know the classes.  The training
phase is also called _fitting_.

Naive Bayes is a classifier that uses Bayes Theorem to ask "what is the
probability that this document belongs to class $C$ (that is, our hypothesis $H$) given the evidence (document) $E$:

$$
\Pr(H\mid E) = \frac{\Pr(E\mid H) \Pr(H)}{\Pr(E)}
$$

Here $H$ is our hypothesis (that the document is spam) and $E$ is the evidence (the document).  The individual terms have names that you
may sometimes see:
* $\Pr(H)$ is the _prior_ probability of $H$
* $\Pr(E\mid H)$ the _likelihood_ of observing $E$ given $H$
* $\Pr(E)$ the _marginal likelihood_ or _model evidence_
* $\Pr(H\mid E)$ the _posterior_ probability of $H$

The ratio of $\Pr(E\mid H) / \Pr(E)$ we call the _impact_ of $E$ on $H$.

In the end, if the evidence $E$ doesn't fit $H$, we reject $H$.
If the evidence $E$ is extremely unlikely, we also will reject $H$ (and Bayes Theorem will tell us this!).  An example is an exceedingly rare disease on which we receive a positive test.  The error in the test (false positive rate) may carry more weight than the test's conclusion.  A more plebian example is the common (yet slightly paranoid) concern that a loved one has befallen some grave ill because he is late arriving.  Often we are Bayesians without realizing it, but our emotions are not!

In [12]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
targets = np.asarray(data['class'])
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's test our classifier.

In [13]:
examples = ['Free Viagra call today!', "I'm going to attend the Linux users group tomorrow."]
example_counts = count_vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions # [1, 0]

array([ 0.,  0.])

Pipelines offer a shortcut in scikit-learn.  We can specify the
set of things we want to do, then do it.  Notice how this is clearer
now that we know what we're doing:

In [16]:
pipeline = Pipeline([
  ('vectorizer',  CountVectorizer()),
  ('classifier',  MultinomialNB()) ])

pipeline.fit(np.asarray(data['text']), np.asarray(data['class']))
pipeline.predict(examples) # [1, 0]


array([ 0.,  0.])

#Cross Validation

Any time we train a model, we must test that it is performing
correctly.  How do we do this, though, unless we have some data set
aside for which we know the correct results?  The solution to that
problem is to set some of our training data aside and to use it for
testing rather than for training.  To make sure we set aside the right
bit of data, we simply try it with different sets.

In one technique, we train on all but one bit of data, then test on the remaining (set aside) bit.  This can take a long time, though, since we have to retrain $N$ times.  Instead, we can set aside $1/k$ of our data and test against it.  This is called $k$-fold cross validation.  Even $k$-fold cross validation is likely to be the longest step in this notebook.

In [20]:
from sklearn.cross_validation import KFold

k_fold = KFold(n=len(data), n_folds=6, indices=False)
scores = []
for train_indices, test_indices in k_fold:
  train_text = np.asarray(data[train_indices]['text'])
  train_y    = np.asarray(data[train_indices]['class'])

  test_text  = np.asarray(data[test_indices]['text'])
  test_y     = np.asarray(data[test_indices]['class'])

  pipeline.fit(train_text, train_y)
  score = pipeline.score(test_text, test_y)
  scores.append(score)

score = sum(scores) / len(scores)
print(score)


0.98862048455


/home/jeff/src/jma/talks/stats-intro/venv/lib/python3.4/site-packages/sklearn/cross_validation.py:69: DeprecationWarning: The indices parameter is deprecated and will be removed (assumed True) in 0.17
  stacklevel=1)


#Trying other models

The pipeline construct lets us easily try other models.  Try these models and then cross-validate again to see the results.  Notice that the cross validation cell fits the data for us.

In [22]:
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
  ('classifier',         MultinomialNB()) ])

In [25]:
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
  ('tfidf_transformer',  TfidfTransformer()),
  ('classifier',         MultinomialNB()) ])
